In [1]:
#https://www.youtube.com/watch?v=8ZZDNd4eyVI&t=1s&ab_channel=Mr.PSolver
import numpy as np
import sympy as smp
import pickle
filename = "variables.pickle"

Symbols Init

In [3]:
g = smp.symbols('g')
M1, M2 = smp.symbols('M1 M2')
L1, L2 = smp.symbols('L1, L2')
T1, T2 = smp.symbols('T1, T2')


KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, d1, d2, currentX, currentY, I1, I2 = smp.symbols('KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, d1, d2, currentX, currentY, I1, I2')

th1, th2 = smp.symbols('th1, th2')
th1d, th2d = smp.symbols('th1d, th2d')
th1dd, th2dd = smp.symbols('th1dd, th2dd')

Dynamic Model

In [51]:

#I1 = 1/12*L1*L1*M1
#I2 = 1/12*L2*L2*M2

#d1 = L1/2
#d2 = L2/2

P1 = smp.Matrix([ - L2*smp.cos(th1 + th2) - L1*smp.cos(th1) ,
                   - L2*smp.sin(th1 + th2) - L1*smp.sin(th1) ])
   
P2 = smp.Matrix([ - L2*smp.cos(th1 + th2) ,
                   - L2*smp.sin(th1 + th2) ])
 
M = smp.Matrix([ [I2 + I1 + 2*M2*d2*L1*smp.cos(th2) + M2*L1*L1 + M2*d2*d2 + M1*d1*d1  , I2 + d2*d2 * M2 + M2*d2*L1*smp.cos(th2)] ,
                  [I2 + d2*M2*L1*smp.cos(th2) + M2*d2*d2                             , I2 + d2*d2 * M2                      ]])
          
B = smp.Matrix([ - d2*M2*L1*th2d*th2d*smp.sin(th2) - 2*M2*d2*L1*th1d*th2d*smp.sin(th2),
                    d2*M2*L1*smp.sin(th2)*th1d*th1d])
              
G = smp.Matrix([ - g*M2*d2*smp.sin(th1 + th2) - (M2*L1 + d1*M1)*g*smp.sin(th1) ,
                  - d2*g*M2*smp.sin(th1 + th2)                              ])
      
T = smp.Matrix([ T1 ,
                  T2 ])

thedd = smp.Matrix([th1dd, th2dd])
  
Zero = M * thedd + B + G - T

sols = smp.solve(Zero, (th1dd, th2dd), simplify=True, rational=False)

th1ddSol = sols[th1dd]
th2ddSol = sols[th2dd]

Impedance Control

In [37]:
J = smp.Matrix([[- L2*smp.cos(th1 + th2) - L1*smp.cos(th1), -L2*smp.cos(th1 + th2)],
                [- L2*smp.sin(th1 + th2) - L1*smp.sin(th1), -L2*smp.sin(th1 + th2)],
                [                                0,                  0]])


desiredPosition = smp.Matrix([desiredX, desiredY, 0 ])
desiredVelocity = smp.Matrix([desiredXdott, desiredYdott, 0])

thed = smp.Matrix([th1d, th2d])

currentPosition = smp.Matrix([currentX,  currentY, 0])

impedanceControl = J.T*(KP*(desiredPosition - currentPosition) + KD*(desiredVelocity - J*thed))

impedanceControl1, impedanceControl2 = impedanceControl 

Save/Pickle variables

In [38]:
stuff = {
    "thdotdot1": th1ddSol,
    "thdotdot2": th2ddSol,
    "impedanceControl1": impedanceControl1,
    "impedanceControl2": impedanceControl2,
    "gravityCompensation1": gravityComp1Sol,
    "gravityCompensation2": gravityComp2Sol
}

with open(filename, 'wb') as f:
    pickle.dump(stuff, f)

Load/Unpickle variables

In [4]:
with open(filename, 'rb') as f:
    stuff = pickle.load(f)
    
th1ddSol = stuff['thdotdot1']
th2ddSol = stuff['thdotdot2']
impedanceControl1 = stuff['impedanceControl1']
impedanceControl2 = stuff['impedanceControl2']
gravityComp1Sol = stuff['gravityCompensation1']
gravityComp2Sol = stuff['gravityCompensation2']

th1ddSol = (2*(I2*T1 - I2*T2 + T1*d2*d2*M2 - T2*d2*d2*M2 + d2*d2*d2*L1*M2*M2*th1d*th1d*smp.sin(th2) + d2*d2*L1*M2*M2*th2d*th2d*smp.sin(th2) + (d2*d2*g*L1*M2*M2*smp.sin(th1))/2 + I2*d1*g*M1*smp.sin(th1) - T2*d2*L1*M2*smp.cos(th2) + I2*g*L1*M2*smp.sin(th1) + (d2*d2*L1*L1*M2*M2*th1d*th1d*smp.sin(2*th2))/2 - (d2*d2*g*L1*M2*M2*smp.sin(th1 + 2*th2))/2 + 2*d2*d2*d2*L1*M2*M2*th1d*th2d*smp.sin(th2) + I2*d2*L1*M2*th1d*th1d*smp.sin(th2) + I2*d2*L1*M2*th2d*th2d*smp.sin(th2) + d1*d2*d2*g*M1*M2*smp.sin(th1) + 2*I2*d2*L1*M2*th1d*th2d*smp.sin(th2)))/(2*I1*I2 + d2*d2*L1*L1*M2*M2 + 2*I2*d1*d1*M1 + 2*I1*d2*d2*M2 + 2*I2*L1*L1*M2 + 2*d1*d1*d2*d2*M1*M2 - d2*d2*L1*L1*M2*M2*smp.cos(2*th2))
th2ddSol = -(T2 - ((M2*d2*d2 + L1*M2*smp.cos(th2)*d2 + I2)*(d2*L1*M2*smp.sin(th2)*th2d*th2d + 2*d2*L1*M2*th1d*smp.sin(th2)*th2d + T1 + g*smp.sin(th1)*(d1*M1 + L1*M2) + d2*g*M2*smp.sin(th1 + th2)))/(M1*d1*d1 + M2*d2*d2 + 2*M2*smp.cos(th2)*d2*L1 + M2*L1*L1 + I1 + I2) + d2*g*M2*smp.sin(th1 + th2) - d2*L1*M2*th1d*th1d*smp.sin(th2))/((M2*d2*d2 + I2)*((M2*d2*d2 + L1*M2*smp.cos(th2)*d2 + I2)*(M2*d2*d2 + L1*M2*smp.cos(th2)*d2 + I2)/((M2*d2*d2 + I2)*(M1*d1*d1 + M2*d2*d2 + 2*M2*smp.cos(th2)*d2*L1 + M2*L1*L1 + I1 + I2)) - 1))


Gravity Compensation

In [108]:
#gravityCompSols = smp.solve([th1ddSol, th2ddSol], (T1, T2), simplify=True, rational=True)

gravityComp1Sol = gravityCompSols[T1]
gravityComp2Sol = gravityCompSols[T2]

gravityComp1Sol = - d2*L1*M2*smp.sin(th2)*th2d*th2d - 2*d2*L1*M2*th1d*smp.sin(th2)*th2d - d2*g*M2*smp.sin(th1 + th2) - d1*g*M1*smp.sin(th1) - g*L1*M2*smp.sin(th1)
gravityComp2Sol = -d2*M2*(- L1*smp.sin(th2)*th1d*th1d + g*smp.sin(th1 + th2))

Turn formulas into functions

In [109]:
thdotdot1_f = smp.lambdify( (T1, T2, g, M1, M2, d1, d2, I1, I2, L1, L2, th1, th2, th1d, th2d) , th1ddSol)
thdotdot2_f = smp.lambdify( (T1, T2, g, M1, M2, d1, d2, I1, I2, L1, L2, th1, th2, th1d, th2d) , th2ddSol)

gravityCompensation1_f = smp.lambdify((g, M1, M2, d1, d2, I1, I2, L1, L2, th1, th2, th1d, th2d), gravityComp1Sol)
gravityCompensation2_f = smp.lambdify((g, M1, M2, d1, d2, I1, I2, L1, L2, th1, th2, th1d, th2d), gravityComp2Sol)

impedanceControl1_f = smp.lambdify((KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, L1, L2, th1, th2, th1d, th2d), impedanceControl1)
impedanceControl2_f = smp.lambdify((KP, KD, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, L1, L2, th1, th2, th1d, th2d), impedanceControl2)

Constants

In [110]:
m1 = 1
m2 = 1
l1 = 1
l2 = 1
g_ = 9.8
Kp = 0
Kd = 0

Forward Kinematics

In [111]:
def forwardKin(the1, the2):
    x = - l2*np.sin(the1 + the2) - l1*np.sin(the1)
    y = l2*np.cos(the1 + the2) + l1*np.cos(the1)
    return x, y

def forwardElbowKin(the1):
    x = -l1*np.sin(the1)
    y = l1*np.cos(the1)
    return x, y

Physics Init

In [125]:
theta1 = np.pi*1/4
theta1dot = 0
theta2 = np.pi*1/4
theta2dot = 0

previous_theta1 = theta1
previous_theta2 = theta2

previous_theta1dot = theta1dot
previous_theta2dot = theta2dot

end_effector_x = 0
end_effector_y = 0
desired_end_effector_x, desired_end_effector_y = forwardKin(theta1, theta2)

t = 0
dt = 0.04

def updateTheta(T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot):
    theta1dotdot = thdotdot1_f(T1_, T2_, g_, m1, m2, l1/2, l2/2, 1/12*l1*l1*m1, 1/12*l2*l2*m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    theta2dotdot = thdotdot2_f(T1_, T2_, g_, m1, m2, l1/2, l2/2, 1/12*l1*l1*m1, 1/12*l2*l2*m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    
    theta1dot = theta1dotdot*dt + theta1dot
    theta2dot = theta2dotdot*dt + theta2dot

    theta1 = theta1dot*dt + theta1
    theta2 = theta2dot*dt + theta2
    return theta1, theta2, theta1dot, theta2dot


def impedenceControl(theta1, theta2, theta1dot, theta2dot):
    
    currentX, currentY = forwardKin(theta1, theta2)
    
    desiredX = desired_end_effector_x
    desiredY = desired_end_effector_y
    desiredXdott = 0
    desiredYdott = 0
    
    
    impedanceControl1 = impedanceControl1_f(Kp, Kd, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, l1, l2, theta1, theta2, theta1dot, theta2dot)
    impedanceControl2 = impedanceControl2_f(Kp, Kd, desiredX, desiredY, desiredXdott, desiredYdott, currentX, currentY, l1, l2, theta1, theta2, theta1dot, theta2dot)
    
    # gravity compensation
    gravityCompensation1 = gravityCompensation1_f(g_, m1, m2, l1/2, l2/2, 1/12*l1*l1*m1, 1/12*l2*l2*m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    gravityCompensation2 = gravityCompensation2_f(g_, m1, m2, l1/2, l2/2, 1/12*l1*l1*m1, 1/12*l2*l2*m2, l1, l2, theta1, theta2, theta1dot, theta2dot)
    print("comp", gravityCompensation1, gravityCompensation2)
    T1_ = impedanceControl1 + gravityCompensation1
    T2_ = impedanceControl2 + gravityCompensation2
    
    return T1_, T2_
    

Init OpenCV

In [126]:
import cv2
from IPython.display import clear_output

cv2.destroyAllWindows()


def forceIntroduced(event, x, y, flags, param):
    global desired_end_effector_x, desired_end_effector_y

    if event==cv2.EVENT_LBUTTONDOWN:
        desired_end_effector_x = x*2/100 - 4
        desired_end_effector_y = -y*2/100 + 4
        print("dx", desired_end_effector_x, desired_end_effector_y, "x", end_effector_x, end_effector_y, "diff", desired_end_effector_x - end_effector_x, desired_end_effector_y - end_effector_y)
        clear_output(wait=True)
        
def kpChanged(arg):
    global Kp
    Kp = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd)

def kdChanged(arg):
    global Kd
    Kd = arg
    clear_output(wait=True)
    print("Kp", Kp, "Kd", Kd)

    
    
cv2.namedWindow('output', cv2.WINDOW_AUTOSIZE)
cv2.createTrackbar('Kp', 'output', 0, 20, kpChanged)
cv2.createTrackbar('Kd', 'output', 0, 20, kdChanged)

cv2.setTrackbarPos('Kp','output', int(Kp))
cv2.setTrackbarPos('Kd','output', int(Kd))


width = 400
height = 400

def draw(theta1, theta2):
    global end_effector_x, end_effector_y
    
    window = np.zeros((height, width, 3), dtype=np.uint8)
    
    
    xElbow, yElbow = forwardElbowKin(theta1)
    xEndEffector, yEndEffector = forwardKin(theta1, theta2)
    
    elbow = (200 + xElbow*70, 200 - yElbow*70)
    end_effector = (200 + xEndEffector*70, 200 - yEndEffector*70)
    
    #elbow = (200 + l1*np.sin(theta1)*70, 200 - l1*np.cos(theta1)*70)
    #end_effector = (elbow[0] + l2*np.sin(theta1 + theta2)*70, elbow[1] - l2*np.cos(theta1 + theta2)*70)

    elbow = (int(elbow[0]), int(elbow[1]))
    end_effector = (int(end_effector[0]), int(end_effector[1]))
    
    end_effector_x = end_effector[0]*2/100 - 4
    end_effector_y = - end_effector[1]*2/100 + 4

    window = cv2.line(window, (200, 200), elbow, (255, 0, 0), 6)
    window = cv2.line(window, elbow, end_effector, (255, 0, 0), 6)
    
    window = cv2.circle(window, (int((desired_end_effector_x+4)*100/2), int((-desired_end_effector_y+4)*100/2)), 5, (255, 255, 255), 2)
    return window

In [127]:
def reset():
    global theta1, theta1dot, theta2, theta2dot, t
    theta1 = np.pi*1/4
    theta1dot = 0
    theta2 = np.pi*1/4
    theta2dot = 0
    t = 0

while True:
    try:
        T1_, T2_ = impedenceControl(theta1, theta2, theta1dot, theta2dot)
        #T1_ = 0
        #T2_ = 0
        theta1, theta2, theta1dot, theta2dot = updateTheta(T1_, T2_, dt, theta1, theta2, theta1dot, theta2dot)
        window = draw(theta1, theta2)
        t = t + dt
        cv2.imshow('output', window)
        cv2.setMouseCallback('output', forceIntroduced)

    except Exception as e:
        print(e)
        reset()

    if cv2.waitKey(1) & 0xFF == ord('r'):
        print("reseting")
        reset()
        
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

        
cv2.destroyAllWindows()

comp -6.536092161127019 -2.532772459427188
comp -6.9945926072674744 -2.7307350784471707
comp -7.244780368782484 -2.8639688348027663
comp -7.533807569116314 -2.9261622865812607
comp -7.920429358396307 -2.935621286791915
comp -8.402754994679992 -2.9124410862944803
comp -8.970770297768034 -2.8746395904673196
comp -9.611471494103878 -2.838126899081386
comp -10.305995272980113 -2.815244710475858
comp -11.026149682410425 -2.810752965441939
comp -11.733855180900987 -2.8176102693331004
comp -12.38717877554401 -2.819854392707602
comp -12.95218192092015 -2.806170151828554
comp -13.413229809163198 -2.7815269692887328
comp -13.775005205317548 -2.7628513410359727
comp -14.055563292499185 -2.7662633227932067
comp -14.275596979794141 -2.8000460856349694
comp -14.45089000321726 -2.8648670874457616
comp -14.590270648323674 -2.956373244425485
comp -14.696882113671272 -3.067428073339564
comp -14.770382344781847 -3.189644974877982
comp -14.809076100406543 -3.3145495591182192
comp -14.811823826453049 -3.43